# In this file, we will define the complete Machine Learning Pipeline to use the Trained Models in the other files to extract the average temperature from the thermal videos

-> Importing the necessary libraries:

In [32]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import joblib

from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2

import warnings
warnings.filterwarnings('ignore')

-> Reading the Data Sheet

In [33]:
dff = pd.read_csv("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/PRML_VOLUNTEER_DATA - Sheet1.csv")

# Loading all 4 Trained Models Namely Convolutional Neural Network, Support Vector Machine, Multi-Layer Perceptron, and Random Forest

-> Extra Libraries for loading the models

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam

1. Multi-Layer Perceptron

In [35]:
model_mlp = Sequential()
model_mlp.add(Dense(512, input_dim=784, activation='relu'))
model_mlp.add(Dense(512, input_dim=784, activation='relu'))
model_mlp.add(Dense(256, activation='relu'))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dense(10, activation='softmax'))

model_mlp.compile('adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'],)

model_mlp.load_weights("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/model-mlp.ckpt")

2. Convolutional Neural Network

In [36]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28,1)))
model_cnn.add(layers.MaxPooling2D((2, 2), strides=2))
model_cnn.add(layers.Conv2D(4, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(10, activation='softmax'))

opt = Adam(learning_rate=0.001)
model_cnn.compile(opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_cnn.load_weights("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/model.ckpt")

3. Random Forest

In [37]:
model_rf = joblib.load("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/RF.joblib")

4. Support Vector Machine

In [38]:
model_svm = joblib.load("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/SVM.joblib")

# Coding all the Helper Functions

# The below function reads the desired image created in the function and then applies filters on the image. Then, we search for contours of defined size and then crop those contours to extract the number images from the desired image

In [39]:
def createNumberImages():

  # Reading the Desired Image
  image = cv2.imread("desired.png")

  # Resizing the image to height of 500
  image = imutils.resize(image, height=500)

  # Converting image to Grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Applying GaussianBlur to the image
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  edged = cv2.Canny(blurred, 50, 254, 255)

  # Finding the contours in the image
  contours, hierarchy = cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # Defining the appropriate area of a contour
  AREA = (image.shape[0]*image.shape[1])/150

  # Counting the number of digits count
  count = 0

  for cnt in contours:
      if (cv2.contourArea(cnt) > AREA):

          # Create a Bounding Box for the contour
          x, y, width, height = cv2.boundingRect(cnt)

          # Cropping the digit from the image
          roi = image[y-20:y+height+20, x-30:x+width+30]

          # Checking if the cropped image exists:
          if (roi.shape[0] > 0 and roi.shape[1] > 0):
            count += 1
            roi = cv2.resize(roi,(28,28))
            cv2.imwrite("roi"+str(count)+".png", roi)
  
  # Returning the number of digits found:
  return count


# The below function predicts the temperature from the digit images that we extracted in the above function

In [40]:
from PIL import Image

def predict(number,typeofmodel):

  # This means that there are postive number of images:
  if (number != -1):

    # Adding a padding of 5 black pixels around each of the digit images so that it is easier to predict by the models:
    for k in range(1,number+1):
      image = Image.open("roi"+str(k)+".png")
        
      right = 5
      left = 5
      top = 5
      bottom = 5
        
      width, height = image.size
        
      new_width = width + right + left
      new_height = height + top + bottom
        
      result = Image.new(image.mode, (new_width, new_height), (0, 0, 0))
        
      result.paste(image, (left, top))
        
      result.save("roi"+str(k)+".png")
    
    # Creating a string to store the temperature predicted by joining all the individual digit predictions:
    finalans = ""

    # Reading the digit after the decimal place:
    img = cv2.imread("roi1.png",0)
    img = cv2.resize(img,(28,28))

    # Transforming the Image to either 0 or 255 for better prediction:
    for i in range(28):
      for j in range(28):
        if (img[i][j] < 127):
          img[i][j] = 0
        else:
          img[i][j] = 255
    
    # Predicting the Digit based on the classifier (given as typeofmodel):
    if (typeofmodel == 'cnn'):

      img = np.reshape(img,(28,28,1))

      img = np.array([img])
 
      ans = np.argmax(model_cnn.predict(img))
    
    elif (typeofmodel == 'rf'):
      img = img.ravel()

      img = np.array([img])

      ans = model_rf.predict(img)[0]
    
    elif (typeofmodel == 'mlp'):
      img = img.ravel()

      img = np.array([img])

      ans = np.argmax(model_mlp.predict(img))
    
    elif (typeofmodel == 'svm'):
      img = img.ravel()

      img = np.array([img])

      ans = model_svm.predict(img)[0]

    # Appending the predicted answer after the decimal point
    finalans = '.' + str(ans)

    # Now predicting all the digits before the decimal point:
    for l in range(2,number+1):
        img = cv2.imread("roi"+str(l)+".png",0)
        img = cv2.resize(img,(28,28))

        # Transforming the Image to either 0 or 255 for better prediction:
        for i in range(28):
          for j in range(28):
            if (img[i][j] < 127):
              img[i][j] = 0
            else:
              img[i][j] = 255

        # Predicting the Digit based on the classifier (given as typeofmodel):
        if (typeofmodel == 'cnn'):
          img = np.reshape(img,(28,28,1))

          img = np.array([img])
        
          ans = np.argmax(model_cnn.predict(img))
        
        elif (typeofmodel == 'rf'):

          img = img.ravel()

          img = np.array([img])

          ans = model_rf.predict(img)[0]
        
        elif (typeofmodel == 'mlp'):

          img = img.ravel()

          img = np.array([img])

          ans = np.argmax(model_mlp.predict(img))
        
        elif (typeofmodel == 'svm'):

          img = img.ravel()

          img = np.array([img])

          ans = model_svm.predict(img)[0]

        if (l == number and ans < 9 and ans != 1):
          ans = 9
        
        if (l == 2 and ans == 3):
          ans = 8

        # Appending the predicted digit to the temperature string
        finalans = str(ans) + finalans 

    # Returning the predicted temperature
    return finalans

# The below function puts together all the above helper function. The below function operates as follows:

*   Firstly, it creates an opencv object to read the video from its address and helps us to access each frame in the video easily
*   We create two arrays to store the predicted temperatures from frames where the bounding boxes are both red and green
*   We loop through each frame in the Video
*   In each frame, the first task is to find the outer blue bounding box and for this we check the hsv values of each pixel fall in the range of blue or not. By doing this, we get the four corners of the outer blue bounding box
*   Now, the next task is to obtain whether inside this blue bounding box whether we have a red bounding box or a green one. Again, we do this by comparing with the hsv values of green and red respectively.
*   In both situations, we create desired images by cropping off necessary portions of the image which just contain the numerical part and try not to take lot of noise.
*   Now, in both cases we predict the temperature in the desired image from the helper function above. We append this predicted image to either of the two arrays created.
*   Finally, if we find green boxes in the video, we return the average values of the green predicted temperatures. Else, we return the average values of the red box frame temperatures







In [41]:
def findFrames(videopath,typeofmodel):

  # Creating an opencv object to read the videos:
  cap = cv2.VideoCapture(videopath)
  ret, frame = cap.read()

  # Creating the arrays for storing the temperatures in each of the situations:
  predicted_green = []
  predicted_red = []

  # Looping through each frame in the video:
  while (cap.isOpened()):
      val = -1

      # These values store the pixel boundaries of the blue bounding box:
      maxbluex = -1
      minbluex = 300
      maxbluey = -1
      minbluey = 400

      # Reading the frame:
      ret, frame = cap.read()
      if ret == False:
          break
      cv2.imwrite('temp.png',frame)
      img = cv2.imread("temp.png")

      # Converting the frame to HSV format:
      hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
      for i in range(300):
        for j in range(400):
          h,s,v = hsv[i][j]

          # Checking if the pixel falls in HSV range for blue color or not:
          if (h >= 110 and h <= 130 and s >= 50 and s <= 255 and v >= 50 and v <= 255):
            if (i > maxbluex):
              maxbluex = i
            if (i < minbluex):
              minbluex = i
            if (j > maxbluey):
              maxbluey = j
            if (j < minbluey):
              minbluey = j

      count_g = 0
      count_r = 0

      # These values store the pixel boundaries of the red and green bounding boxes:

      maxgreenx = minbluex 
      mingreenx = maxbluex 
      maxgreeny = minbluey 
      mingreeny = maxbluey 

      maxredx = minbluex 
      minredx = maxbluex 
      maxredy = minbluey 
      minredy = maxbluey 

      # Looping through the area inside the blue bounding box:
      for i in range(minbluex,maxbluex):
        for j in range(minbluey,maxbluey):
          h,s,v = hsv[i][j]

          # Checking if the pixel lies in range for green color:
          if (h >= 65 and h <= 80 and s >= 60 and s <= 255 and v >= 60 and v <= 255):
            count_g += 1
            if (i > maxgreenx):
              maxgreenx = i
            if (i < mingreenx):
              mingreenx = i
            if (j > maxgreeny):
              maxgreeny = j
            if (j < mingreeny):
              mingreeny = j
          
          # Checking if the pixel lies in range for red color:
          if (h >= 150 and h <= 190 and s >= 140 and s <= 255 and v >= 80 and v <= 255):
            count_r += 1
            if (i > maxredx):
              maxredx = i
            if (i < minredx):
              minredx = i
            if (j > maxredy):
              maxredy = j
            if (j < minredy):
              minredy = j
      
      # This means that there is a green bounding box:
      if (count_g > 10):

        # Creating the desired image:
        img = img[mingreenx-25:mingreenx + int((maxgreenx-mingreenx)/4.5),mingreeny-10:maxgreeny+25,:]
        desiredimg = img
        cv2.imwrite('desired.png',img)

        # Obtaining the number of digits in the desired image:
        xx = int(createNumberImages())

        # Obtaining the predicted temperature and appending to the respective array:
        if (xx >= 3):
          anss = predict(xx,typeofmodel)
          if (float(anss) < 110):
            predicted_green.append(anss)
      
      # This means that there is a red bounding box:
      elif (count_r > 10 and len(predicted_red) < 10):
        
        # Creating the desired image:
        img = img[minredx-25:minredx + int((maxredx-minredx)/4.5),minredy-10:maxredy+25,:]
        desiredimg = img
        cv2.imwrite('desired.png',img)

        # Obtaining the number of digits in the desired image:
        xx = int(createNumberImages())

        # Obtaining the predicted temperature and appending to the respective array:
        if (xx >= 3):
          anss = predict(xx,typeofmodel)
          if (float(anss) < 110):
            predicted_red.append(anss)
      
      # Stopping condition:
      if (len(predicted_green) == 10):
        break

  cap.release()
  cv2.destroyAllWindows()

  # Returing the Predicted Temperature in respective conditions:
  if (len(predicted_green) > 0):
    predicted_green = [float(i) for i in predicted_green]
    return np.mean(predicted_green)
  
  if (len(predicted_red) > 0):
    predicted_red = [float(i) for i in predicted_red]
    return np.mean(predicted_red)

  return -1


# The below function uses the above function to obtain the predicted average temperature from a video in a videopath

In [48]:
def temperature_Predict(videopath):
 
  # Defining the models available:
  models = ['cnn','rf','mlp','svm']

  for i in models:

    # Obtaining the prediction:
    ansss = findFrames(videopath,i)

    if ansss != -1:
      print("The Predicted Average Temperature in the Video with "+i+" is " + str(ansss))

    else:
      print("Error in Predicting with "+ i)
      print("Breaking ...")
      break

# Predicting the Temperature in the Desired Video whose address needs to be defined in the path variable:

In [52]:
# User can add custom path here:

path = "/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/PRML_THERMAL_DATA/62.mp4"

temperature_Predict(path)

The Predicted Average Temperature in the Video with cnn is 97.36999999999999
The Predicted Average Temperature in the Video with rf is 99.42999999999999
The Predicted Average Temperature in the Video with mlp is 99.42999999999999
The Predicted Average Temperature in the Video with svm is 98.81


# Displaying the Obtained Results

# So, we extracted the temperatures from many videos in the given dataset. We have obtained the following results from them

In [50]:
df_submission = pd.read_csv("/content/drive/MyDrive/Trisem 3/PR and ML/Mini Project/outputFile.csv")

df_submission

,Video Number,Convolutional Neural Network,Random Forest,Multi-Layer Perceptron,Support Vector Machine,True Value From Sheet
0,31,97.590000,92.16000,97.020000,96.980000,97.5
1,43,98.100000,92.45000,97.900000,97.900000,97.9
2,44,98.000000,93.67500,96.850000,96.450000,98.1
3,47,98.500000,95.53000,98.530000,95.500000,98.4
4,50,98.350000,95.35000,98.350000,95.300000,98.2
5,52,98.470000,95.47000,98.470000,95.470000,98.4
6,53,97.790000,91.83000,96.030000,97.490000,97.6
7,60,98.433333,91.80000,93.800000,97.400000,99.5
8,62,97.370000,99.43000,99.430000,98.810000,99.7
9,63,98.400000,94.40000,97.400000,97.300000,98.5


# We can see that the CNN Model gave closest results to the True Values. It was followed in performance by Support Vector Machine, the the Multi-Layer Perceptron and finally the Random Forest Model performed poorly.

# Thus, in this notebook we have implemented the complete Machine Learning Pipeline which started from reading the Video from its address, then extracting individual digits using various processing techniques and finally predicting the temperatures using the various trained models